<a href="https://colab.research.google.com/github/phenix-project/Colabs/blob/main/AlphaFoldPhenixGUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@markdown <a href="https://phenix-online.org/"><img src="https://phenix-online.org/phenixwebsite_static/mainsite/img/phenix-logo-1.jpg" height="300" align="right" ></a>
#@markdown <font color='black'><h3 style="font-family:verdana;"><-- Click<i> <b>Run cell</b> </i>button to start</font>
#@markdown
#@markdown
#@markdown <b><font color='green'><h1 style="font-family:verdana;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  AlphaFold from the Phenix GUI</font></b>
#@markdown
#@markdown <b><font color='black'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1. Hit the "Run cell" button  to start </font></b>
#@markdown
#@markdown <b><font color='blue'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2. When the "Choose files" button appears, upload your xxx_cycle_x.tgz file </b>
#@markdown
#@markdown <b><font color='blue'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(usually in your Downloads folder) </font></b>
#@markdown
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The "Run cell" button will have a circling arc while running
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;A summary of what is happening will be printed.
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;When finished the "Run cell" button should return to a triangle.
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The output xxxx_x_cycle_x.result.zip file(s) should automatically download.
#@markdown
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; If not, use the file browser to the left to find them and
#@markdown click on the 3 dots to download.
#@markdown
#@markdown <font color='black'><h3 style="font-family:verdana;"> ---> <b><i>Choose files </b></i>button will appear below here <---</font>


from google.colab import files
from pathlib import Path
import os

upload_dir = "/content/upload_dir"
content_dir = "/content"
data_dir = "/content"
input_directory = "upload_dir"
cif_dir = "/content/mmcif"

os.chdir(content_dir)
if not os.path.isdir(cif_dir):
  os.mkdir(cif_dir)

if not os.path.isdir(upload_dir):
  print("Creating upload directory %s" %(upload_dir))
  os.mkdir(upload_dir)
os.chdir(upload_dir)
for fn in os.listdir("."):
  if str(fn).endswith("_sequence_info.dat") or \
     str(fn).startswith("run_") or str(fn).endswith(".tgz"):
    os.remove(fn)  # remove any control files

found_tgz = False
uploaded = files.upload()
for filename,contents in uploaded.items():
  if str(filename).endswith(".tgz"):
    found_tgz = True
    break
if not found_tgz:
  print("Could not find a .tgz file in upload...please try again")
  raise AssertionError ("Please upload a .tgz file")

print("Unpacking .tgz file")
import tarfile
tar = tarfile.open(filename, "r:gz")
tar.extractall()
tar.close()
!ls -tlr
os.chdir(content_dir)

file_with_jobname_sequence_lines = None
file_with_instructions = None
for fn in os.listdir(upload_dir):
  if fn.startswith("run_"):
    file_with_instructions = fn
  if fn.endswith("_sequence_info.dat"): 
    file_with_jobname_sequence_lines = os.path.join(upload_dir,fn)
if not file_with_jobname_sequence_lines:
  text = "Unable to find a file_with_jobname_sequence_lines in %s" %(
      upload_dir)
  print(text)
  raise AssertionError (text)
if not file_with_instructions:
  text = "Unable to find a file_with_instructions in %s" %(
      upload_dir)
  print(text)
  raise AssertionError (text)

# Get instructions from file_with_instructions
text = open(os.path.join(upload_dir,file_with_instructions)).read()
instructions = text.split()

number_of_models = 1
include_templates_from_pdb=False
random_seed=86812
upload_manual_templates=False
maximum_templates_from_pdb = 20
random_seed = 581867
skip_all_msa = False

for arg in instructions:
  if arg.find("=")> -1:
    kw,value = arg.split("=")
    if kw == "number_of_models":
      number_of_models = int(value)
    elif kw == "include_templates_from_pdb": 
      if value.lower()=='true':
        include_templates_from_pdb = True
      else:
        include_templates_from_pdb = False
    elif kw == "template_file":
      upload_manual_templates = True
    elif kw == "maximum_templates_from_pdb":
      maximum_templates_from_pdb = int(value)
    elif kw == "random_seed":
      random_seed = int(value)
    elif kw == "skip_all_msa":
      if value.lower() == 'true':
        skip_all_msa = True
      else:
        skip_all_msa = False

sequence = ''
resolution =  None
jobname = 'None'
phenix_download_password='None'
query_sequence = sequence
password = phenix_download_password

# Save all parameters in a dictionary
params = {}
for p in ['cif_dir','content','resolution','jobname', 'password', 'query_sequence']:
  params[p] = locals().get(p,None)

save_outputs_in_google_drive = False 

uploaded_templates_have_exact_sequence = True 
uploaded_templates_are_map_to_model = False
maximum_cycles =  1
phenix_version = None
version = phenix_version  # rename variable

upload_file_with_jobname_sequence_lines = False
upload_file_with_jobname_resolution_sequence_lines = upload_file_with_jobname_sequence_lines

if file_with_jobname_sequence_lines and input_directory and os.path.isfile(
    os.path.join(input_directory,file_with_jobname_sequence_lines)):
  file_with_jobname_sequence_lines = os.path.join(input_directory,file_with_jobname_sequence_lines)
  print("FILE WITH JOBNAME SEQUENCE LINES:",file_with_jobname_sequence_lines)
else:
  print("FILE WITH JOBNAME SEQUENCE LINES MISSING:",file_with_jobname_sequence_lines)
  file_with_jobname_sequence_lines = None
file_with_jobname_resolution_sequence_lines = file_with_jobname_sequence_lines
msa_use = 'Use MSA throughout' 

random_seed_iterations =  number_of_models 

debug = True 

params['jobname'] = None
params['resolution'] = None
params['sequence'] = None


skip_all_msa_after_first_cycle = False

upload_maps = False  # Always for this version
upload_msa_file = False  # Always for this version
starting_alphafold_model = False  # Always for this version
use_msa = (not skip_all_msa)

minimum_random_seed_iterations = int(max(1,random_seed_iterations//20))

if type(params) != type({}):  # convert back to dict
  params = params()

# Save parameters
for p in ['content_dir','data_dir','save_outputs_in_google_drive','input_directory',
    'include_templates_from_pdb','maximum_templates_from_pdb',
    'upload_manual_templates','uploaded_templates_are_map_to_model',
    'maximum_cycles','version',
    'upload_file_with_jobname_resolution_sequence_lines',
    'file_with_jobname_resolution_sequence_lines',
    'use_msa','skip_all_msa_after_first_cycle',
    'upload_maps','debug','carry_on','random_seed',
    'upload_msa_file','starting_alphafold_model',
    'random_seed_iterations','minimum_random_seed_iterations']:
  params[p] = locals().get(p,None)

print("\nINSTALLING JAX 0.3.15")
!pip install jax==0.3.15
!pip install jaxlib==0.3.15
print("\nDONE INSTALLING JAX 0.3.15")
print("\nPLEASE IGNORE ALL jax ERROR MESSAGES")

file_name = 'phenix_colab_utils.py'
if os.path.isfile(file_name):
  os.remove(file_name)
os.environ['file_name'] = file_name
result = os.system("wget -qnc https://raw.githubusercontent.com/phenix-project/Colabs/main/alphafold2/$file_name")

import phenix_colab_utils as cu

cu.install_miniconda()

cu.get_helper_files()  # get all the other helper files
cu.clear_python_caches()


# Set up the inputs using the helper python files
from phenix_alphafold_utils import set_up_input_files
params = set_up_input_files(params, convert_to_params = False)
params['cif_dir'] = None # ZZZ


import phenix_colab_utils as cu


# Get tensorflow import before installation
if not locals().get('tf'):
  tf = cu.import_tensorflow()

# Install selected software
cu.install_software(
  bioconda = True,
  phenix = False,
    phenix_version = params.get('version'),
    phenix_password = params.get('password'),
  alphafold = True,
  pdb_to_cif = True
    )

print("\nRUNNING ALPHAFOLD NOW...")
# Convert params from dict to alphafold_with_density_map params
from phenix_alphafold_utils import get_alphafold_with_density_map_params
params = get_alphafold_with_density_map_params(params)

from run_alphafold_with_density_map import run_jobs

# Working directory
os.chdir(params.content_dir)
results = run_jobs(params)

print("\nALL DONE")


Saving phenix6L5L_16_cycle_1.tgz to phenix6L5L_16_cycle_1.tgz
Unpacking .tgz file
total 16
-rw-rw-rw- 1 root root 619 Sep 13 13:19 run_phenix6L5L_16_predicted_1_cycle_1.dat
-rw-rw-rw- 1 root root 400 Sep 13 13:19 phenix6L5L_16_cycle_1_sequence_info.dat
-rw-rw-rw- 1 root root 372 Sep 13 13:19 phenix6L5L_16_1_cycle_1_input.fa
-rw-r--r-- 1 root root 780 Sep 14 11:46 phenix6L5L_16_cycle_1.tgz
FILE WITH JOBNAME SEQUENCE LINES: /content/upload_dir/phenix6L5L_16_cycle_1_sequence_info.dat

INSTALLING JAX 0.3.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72.0 MB 40 kB/s 

DONE INSTALLING JAX 0.3.15

PLEASE IGNORE ALL jax ERROR MESSAGES
Clearing python caches ...
Input files will be taken from upload_dir
Input directory:  upload_dir
Output directory:  /content
Upload dir will be: /content/upload_dir
Read

AssertionError: ignored

### <center> <b> <font color='black'>  Helpful hints </font></b> </center>

1. TYPICAL JOB: A normal job should just consist of hitting the Run cell button, saying ok to any questions from Colab and uploading the .tgz file from your Downloads folder.

2. STARTING UP and the RUN CELL icon. Start your run by hitting the <b><i>Run cell</i></b> button at the upper left corner of the notebook (looks like a white triangle in a black circle). When it starts, the white triangle turns into a square and the circle has a rotating black arc circling around it.  If it stops, either the white triangle comes back (ok finish), or the icon turns red (a crash).

3. UPLOADING. When the upload button appears, upload your .tgz file (usually in your Downloads directory) as specified by the Phenix GUI.  If your jobname (in th Phenix GUI) is my_job then this file will have a name like "my_job_cycle_3.tgz" if this is cycle 3 of building. The Phenix GUI window that had the button to get to this notebook will show the name of this file.

4. STOPPING A RUN. You can stop the run by clicking on the "Run cell" button. It will turn red.  You can rerun it by clicking it again. This will allow you to upload the .tgz file again (or upload another one).

5. CRASHES. If something goes wrong, the "Run cell" button will turn red and a message should appear in red. You can hit the "Run cell" icon to run again. 

6. GPU and HIGH RAM. At the upper right of the notebook to the left of the "Editing" icon there is a little down-arrow. This has a "view resources" option that allows you to change the "runtime type" if necessary. You want Hardware accelerator set to GPU and Runtime shape set to High-RAM if possible (may not be possible if you have standard Colab, so Standard is ok but may not be sufficient for larger chains). Background execution will usually be off (but if you have Colab Pro+ it may be on, allowing you to keep running while you are disconnected).

7. RAM, DISK and RECONNECT. Also to the left of the "Editing" icon there will be either two progress bars labelled RAM and DISK or the word "Reconnect".  The RAM and DISK icons tell you how much of your allocated memory and disk space are used.  If "Reconnect" shows, it means that you are not connected to a "Runtime" (a computer that will run your notebook).  You can hit "Reconnect" and it should try to connect you.

8. BUSY. If the Colab computers are crowded, you can get a "Busy" when you try to "Reconnect" or when you hit the "Run cell" button to try and run.  In this case you may have to wait and try again (try a few times, often it will start up by itself eventually once you hit "Reconnect", but you might have to wait a few hours too.)

9. IS IT RUNNING. Look for the rotating black arc around the Run cell icon and the RAM and DISK icons (and make sure it does not say "Busy" at the upper right to the left of the Editing icon).

10. DID IT INCLUDE TEMPLATES. Look in the text that is printed for something like: "Including templates:
	 MY_JOB_1_CYCLE_3_INPUT_A".  Here the "1" is chain 1 and the "3" is cycle 3 and the jobname is my_job.

11. AUTOMATIC DOWNLOAD. When the notebook finishes it should download a zip file with your results. If necessary, copy this file to the directory specified in the Phenix GUI. The name of this zip file if your jobname is "my_job" and this is cycle 3 of building chain 1 will be "my_job_1_cycle_3.result.zip".  Usually the automatic download puts the zip file in the right place and your Phenix GUI job continues on by itself without you having to do anything.

12. CLOSE NOTEBOOK WHEN DONE. You can close this notebook when done.

13. MULTIPLE CHAINS.  If your structure has multiple chains, then each one will be predicted separately and each one will get its own downloaded zip file.

14. GENERALLY IGNORE WARNINGS.  You may get a warning like "FutureWarning: jax.tree_util.tree_multimap() is deprecated".  This can be ignored.

15. UNABLE TO FIND FILE:  If you get the error: "Unable to find a file_with_instructions in /content/upload_dir" this means that the file you uploaded (your .tgz file) did not contain everything that was expected.  It usually means that this was the wrong file to upload (just hit Run cell again). It could also be that the Phenix GUI did not write out all the necessary files. (If you unpack the .tgz file it should contain a file with a name like run_jobname_predicted_1_cycle_3.dat).

<b> <font color='black'> <center>Please cite the ColabFold and AlphaFold2 papers if you use this notebook:</center>
</font></b> 

- <font color='green'>[Mirdita, M.,  Ovchinnikov, S., Steinegger, M.(2021). ColabFold - Making protein folding accessible to all *bioRxiv*, 2021.08.15.456425](https://www.biorxiv.org/content/10.1101/2021.08.15.456425v2)</font> 

- <font color='green'> [Jumper, J., Evans, R., Pritzel, A. et al. Highly accurate protein structure prediction with AlphaFold. Nature 596, 583–589 (2021)](https://www.nature.com/articles/s41586-021-03819-2)
</font>
-----------------
